In [1]:
import json
import numpy as np

file = open('intents.json').read()
file = json.loads(file)#loading the json file which has our intents

In [2]:
import nltk

words=[]
document=[]
classes = []
for intent in file['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)# Change pattern into words 2D array
        words.extend(w)#convert word into one array
        document.append((w,intent['tag']))#append tag with sentence like (['Hi', 'there'], 'greeting')
        if intent['tag'] not in classes:
            classes.append(intent['tag'])#saving all the tags

In [3]:
import pickle
from nltk.stem import WordNetLemmatizer

lemmatizer=WordNetLemmatizer()#WordNetLemmatizer used convert any word into actual form like believes into belief

ignore_word = ['?','!']
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_word]#removing ignore word and converting into lower and origninal form of word

words = sorted(list(set(words)))#sorting words and removing duplicates
classes = sorted(list(classes))

pickle.dump(words,open('words.pickle','wb'))#wb means writing in binary mode
pickle.dump(classes,open('classes.pkl','wb'))
#When we load use pickle.load(open('words.pickle','rb')) reading in binary

In [4]:
#Creating Training set
import random
training = []
output_empty = [0] * len(classes)#making a empty list with size of classes

for doc in document:
    pattern_word = doc[0]#taking pattern form the documents
    bag=[]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_word]#converting pattern into actual form
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)#what bag do make value one whenever word found in words array
        #for making the training features
    output_row = list(output_empty)
    #print(classes.index(doc[1]))
    output_row[classes.index(doc[1])] = 1
    #making labels
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
np.array(train_x).shape

(47, 88)

In [5]:
#Making a model
import tensorflow
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.optimizers import SGD

#making callback for early stopping
class mycallback(tensorflow.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs):
        if (logs.get('acc')>0.999):
            print('\n accuracy is more than 99.9% so cancelling training')
            self.model.stop_training=True
            
model = Sequential([
    Dense(128,input_shape=[len(train_x[0]),],activation='relu'),
    Dropout(0.5),
    Dense(64,activation='relu'),
    Dropout(0.5),
    Dense(len(train_y[0]),activation='softmax')
])
#we want some kind of ‘moving’ average which would ‘denoise’ the data and bring it closer to the original function. thats why we used momentum
model.compile(loss='categorical_crossentropy',optimizer=SGD(lr=0.1, momentum=0.9),metrics=['acc'])#I used momentum = 0.9 above. It is a good value and most often used in SGD with momentum
model.fit(np.array(train_x),np.array(train_y),epochs=200,callbacks=[mycallback()])
model.save_weights('my_model_weights.h5')

Epoch 1/200
47/47 [==============================] - 0s 6ms/sample - loss: 2.2313 - acc: 0.1064
Epoch 2/200
47/47 [==============================] - 0s 161us/sample - loss: 2.2220 - acc: 0.0851
Epoch 3/200
47/47 [==============================] - 0s 189us/sample - loss: 2.1347 - acc: 0.2340
Epoch 4/200
47/47 [==============================] - 0s 194us/sample - loss: 2.0368 - acc: 0.2766
Epoch 5/200
47/47 [==============================] - 0s 174us/sample - loss: 2.0617 - acc: 0.2766
Epoch 6/200
47/47 [==============================] - 0s 202us/sample - loss: 1.8097 - acc: 0.4894
Epoch 7/200
47/47 [==============================] - 0s 200us/sample - loss: 1.7713 - acc: 0.5319
Epoch 8/200
47/47 [==============================] - 0s 212us/sample - loss: 1.5365 - acc: 0.6383
Epoch 9/200
47/47 [==============================] - 0s 186us/sample - loss: 1.3953 - acc: 0.6383
Epoch 10/200
47/47 [==============================] - 0s 231us/sample - loss: 1.1575 - acc: 0.5745
Epoch 11/200
47/47 [=

In [6]:
def get_response(text,model):
    text = nltk.word_tokenize(text)
    
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in text]
    bag=[]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    y_pred = model.predict(np.array([bag]))
    y = np.argmax(y_pred)
    tag = classes[y]
    print(tag)
    intent = file['intents']
    for i in intent:
        if(i['tag'] == tag):
            result = random.choice(i['responses'])
            break
    return result

In [7]:
new_model = model = Sequential([
    Dense(128,input_shape=[len(train_x[0]),],activation='relu'),
    Dropout(0.5),
    Dense(64,activation='relu'),
    Dropout(0.5),
    Dense(len(train_y[0]),activation='softmax')
])

new_model.load_weights('my_model_weights.h5')

In [8]:
get_response('HEllO,How are you?',new_model)

greeting


'Hi there, how can I help?'